In [1]:
import tensorflow as tf

import tensorflow_datasets as tfds
import keras
from keras import layers

In [2]:
# (training_set, validation_set), info = tfds.load('KolektorSDD2TF',
#     split=['train[:70%]', 'train[70%:]'],
#     with_info=True,
#     as_supervised=True,
# )

(test_set, valid_set, train_set), info = tfds.load("KolektorSDD2TF", 
                                           split=["test", "train[0%:30%]", "train[30%:]"],
                                           as_supervised=True, with_info=True)

print("Train set size: ", len(train_set)) # Train set size:  40000
print("Test set size: ", len(test_set))   # Test set size:  10000
print("Valid set size: ", len(valid_set)) # Valid set size:  10000

Train set size:  1632
Test set size:  1004
Valid set size:  699


In [3]:
#image dimensions, seems to work with non-square inputs
number_channels = 3

IMAGE_HEIGHT =  512 #194 # 636 # 388
IMAGE_WIDTH = 128 # 70 # 229 # 140

In [4]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
print(TRAIN_LENGTH)

2331


In [8]:
# Preprocess your dataset (e.g., resize images, normalize pixel values)
def preprocess_image(datapoint):
    input_image = tf.image.resize(datapoint['image'], (IMAGE_HEIGHT, IMAGE_WIDTH))
    input_image /= 255.0
    input_mask = tf.image.resize(datapoint['labeledImage'], (IMAGE_HEIGHT, IMAGE_WIDTH),
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return input_image, input_mask

# Apply preprocessing to the dataset
train_set = train_set.map(preprocess_image)
valid_set = valid_set.map(preprocess_image)
test_set = test_set.map(preprocess_image)

# Combine input images and segmentation masks into tuples
train_set = train_set.map(lambda image, mask: (image, mask))
valid_set = valid_set.map(lambda image, mask: (image, mask))
test_set = test_set.map(lambda image, mask: (image, mask))

# Batch and shuffle the datasets
BATCH_SIZE = 32
train_set = train_set.batch(BATCH_SIZE).shuffle(buffer_size=1000)
valid_set = valid_set.batch(BATCH_SIZE)
test_set = test_set.batch(BATCH_SIZE)

# Optionally prefetch data for improved performance
train_set = train_set.prefetch(tf.data.experimental.AUTOTUNE)
valid_set = valid_set.prefetch(tf.data.experimental.AUTOTUNE)
test_set = test_set.prefetch(tf.data.experimental.AUTOTUNE)


TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__preprocess_image() takes 1 positional argument but 3 were given


In [5]:
#Normalizing inputs necessary
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  # input_mask /= (-255)
  return input_image, input_mask

In [6]:
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (IMAGE_HEIGHT, IMAGE_WIDTH))
  input_mask = tf.image.resize(datapoint['labeledImage'], (IMAGE_HEIGHT, IMAGE_WIDTH),
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
  )
  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [7]:
train_images = train_set.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
val_images = valid_set.map(load_image,  num_parallel_calls=tf.data.AUTOTUNE)
test_images = test_set.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__load_image() takes 1 positional argument but 3 were given


In [55]:
train_batches = (
    train_images
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_images.batch(BATCH_SIZE)

In [35]:
#Building U-net model
#Downward stream
inputs = tf.keras.layers.Input((IMAGE_HEIGHT, IMAGE_WIDTH,number_channels))
conv_11 = layers.Conv2D(64,kernel_size=(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(inputs)
conv_12 = layers.Conv2D(64,kernel_size=(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_11)

max_pool_1 = layers.MaxPool2D((2,2))(conv_12)
conv_21 = layers.Conv2D(128,(3,3),activation = 'relu',padding= 'same',kernel_initializer = 'he_normal')(max_pool_1)
conv_22 = layers.Conv2D(128,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_21)

max_pool_2 = layers.MaxPool2D((2,2))(conv_22)
conv_31 = layers.Conv2D(256,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_2)
conv_32 = layers.Conv2D(256,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_31)

max_pool_3 = layers.MaxPool2D((2,2))(conv_32)
conv_41 = layers.Conv2D(512,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_3)
conv_42 = layers.Conv2D(512,(3,3),activation = 'relu', padding= 'same')(conv_41)

max_pool_4 = layers.MaxPool2D((2,2))(conv_42)
conv_51 = layers.Conv2D(1024,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(max_pool_4)
conv_52 = layers.Conv2D(1024,(3,3),activation = 'relu', padding= 'same',kernel_initializer = 'he_normal')(conv_51)


In [36]:
#Upward stream
upconv_1 = layers.Conv2DTranspose(512,(2,2), strides=(2,2))(conv_52)
upconv_1_conc = layers.concatenate([upconv_1,conv_42])
conv_61 = layers.Conv2D(512,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_1_conc)
conv_62 = layers.Conv2D(512,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_61)

upconv_2 = layers.Conv2DTranspose(256, (2,2), strides = (2,2))(conv_62)
upconv_2_conc = layers.concatenate([upconv_2, conv_32])
conv_71 = layers.Conv2D(256,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_2_conc)
conv_72 = layers.Conv2D(256,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_71)

upconv_3 = layers.Conv2DTranspose(128,(2,2), strides=(2,2))(conv_72)
upconv_3_conc = layers.concatenate([upconv_3,conv_22])
conv_81 = layers.Conv2D(128,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_3_conc)
conv_82 = layers.Conv2D(128,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_81)

upconv_4 = layers.Conv2DTranspose(64,(2,2), strides=(2,2))(conv_82)
upconv_4_conc = layers.concatenate([upconv_4,conv_12])
conv_91 = layers.Conv2D(64,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(upconv_4_conc)
conv_92 = layers.Conv2D(64,(3,3),activation = 'relu', padding = 'same',kernel_initializer = 'he_normal')(conv_91)
outputs = layers.Conv2D(1,(1,1), activation = 'sigmoid', padding = 'same',kernel_initializer = 'he_normal')(conv_92)




In [37]:
#Compiling model
model = tf.keras.Model(inputs = [inputs],outputs = [outputs])
model.compile(optimizer='adam',loss='binary_crossentropy', metrics='accuracy')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 512, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_96 (Conv2D)             (None, 512, 128, 64  1792        ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_97 (Conv2D)             (None, 512, 128, 64  36928       ['conv2d_96[0][0]']              
                                )                                                           